In [5]:
import numpy as np
X = np.random.rand(15,10)
from sklearn.decomposition import NMF
factorizer = NMF(n_components=5,init='random')
W = factorizer.fit_transform(X)
H = factorizer.components_

In [8]:
W.shape, H.shape

((15, 5), (5, 10))

In [46]:
import pandas as pd

In [47]:
df = pd.read_csv('Whole_dataset1.txt')

In [48]:
df = df.sample(n=100)

In [49]:
import numpy as np
from scipy.sparse import csr_matrix

r = np.array(df.CustomerID); no_row = len(set(r))
c = np.array(df.MovieID); no_col = len(set(c))
data = np.array(df.Score)

del df

unique_cust = list(set(r)); unique_movie = list(set(c))
unique_cust.sort(); unique_movie.sort()
import pickle
with open("unique_cust_list.txt","wb") as bi:
    pickle.dump(unique_cust,bi)
bi.close()
new_cust_id = 1; new_movie_id = 1

for i in unique_cust:
    pos = np.where(r == i)[0].tolist()
    for p in pos:
        r[p] = new_cust_id
    new_cust_id += 1

for i in unique_movie:
    pos = np.where(c == i)[0].tolist()
    for p in pos:
        c[p] = new_movie_id
    new_movie_id += 1

Ratings = csr_matrix((data, (r, c)),shape=(no_row+1,no_col+1))

In [51]:
Ratings

<101x89 sparse matrix of type '<class 'numpy.int64'>'
	with 100 stored elements in Compressed Sparse Row format>

In [39]:
to_array = Ratings.toarray()

In [40]:
r,c = np.where(to_array!=0)

In [45]:
Ratings[5,86]

3

In [44]:
print (r);print (c)

[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100]
[72 27 38 30 86 65 53  2 27 25 21  6 39 54 66 35 72 75  1 67  2 52 18 19 42
 23 62 74 44 49 63 45 84 87 51 76  3 39  8 33 90 10 77 41 68 82 85 79 91  5
 31 13 80 70 89  4 12 83 43 16 37 14 61 71 40 89 73 29 36 56 88 48 28 20 46
 59 78 32 22 47 34 80 11 57 26 60 90 15 64 50 58  9  7 46 17 55 69 24 85 81]


In [42]:
ratings_retireval = [to_array[r[i]][c[i]] for i in range(len(r))]

In [43]:
print (ratings_retireval)

[4, 4, 1, 3, 3, 4, 4, 4, 3, 3, 4, 5, 4, 4, 3, 1, 3, 4, 5, 4, 4, 2, 3, 4, 4, 3, 4, 5, 4, 4, 1, 4, 3, 5, 3, 5, 5, 4, 5, 4, 3, 3, 4, 4, 3, 2, 4, 3, 4, 2, 2, 1, 5, 3, 5, 4, 4, 4, 1, 3, 5, 5, 3, 5, 4, 4, 3, 5, 1, 4, 3, 4, 3, 4, 3, 5, 3, 5, 3, 4, 5, 5, 4, 3, 4, 4, 5, 5, 5, 4, 3, 3, 5, 4, 5, 3, 3, 3, 4, 5]


## Matrix Factorization code

In [11]:
import numpy

In [15]:
def matrix_factorization(R, P, Q, K, steps=5000, alpha=0.0002, beta=0.02):
    Q = Q.T
    for step in range(steps):
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    eij = R[i][j] - numpy.dot(P[i,:],Q[:,j])
                    for k in range(K):
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])
        eR = numpy.dot(P,Q)
        e = 0
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    e = e + pow(R[i][j] - numpy.dot(P[i,:],Q[:,j]), 2)
                    for k in range(K):
                        e = e + (beta/2) * ( pow(P[i][k],2) + pow(Q[k][j],2) )
        if e < 0.001:
            break
    return P, Q.T

###############################################################################

if __name__ == "__main__":
    R = [
         [5,3,0,1],
         [4,0,0,1],
         [1,1,0,5],
         [1,0,0,4],
         [0,1,5,4],
        ]

    R = numpy.array(R)

    N = len(R)
    M = len(R[0])
    K = 2

    P = numpy.random.rand(N,K)
    Q = numpy.random.rand(M,K)

    nP, nQ = matrix_factorization(R, P, Q, K)

In [19]:
print (numpy.dot(nP,nQ.T))

[[ 5.04615188  2.79161974  5.53904403  0.99787317]
 [ 3.92901788  2.1754527   4.43837058  0.99995814]
 [ 1.12221196  0.66020895  3.90349335  4.96615451]
 [ 0.94003153  0.55152286  3.16753226  3.97834663]
 [ 2.61334275  1.47488667  4.84545181  4.02718934]]


In [20]:
Ratings.T

<93x101 sparse matrix of type '<class 'numpy.int64'>'
	with 100 stored elements in Compressed Sparse Column format>

In [23]:
Ratings.shape

(101, 93)

## Sparse matrix factorization using sklearn

## Sparse matrix Factorization

In [29]:
def matrix_factorization(R, P, Q, K, steps=5000, alpha=0.0002, beta=0.02):
    Q = Q.T
    for step in range(steps):
        for i in range(Ratings.shape[0]):
            for j in range(Ratings.shape[1]):
                if R[i,j] > 0:
                    eij = R[i,j] - numpy.dot(P[i,:],Q[:,j])
                    for k in range(K):
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])
        eR = numpy.dot(P,Q)
        e = 0
        for i in range(Ratings.shape[0]):
            for j in range(Ratings.shape[1]):
                if R[i,j] > 0:
                    e = e + pow(R[i,j] - numpy.dot(P[i,:],Q[:,j]), 2)
                    for k in range(K):
                        e = e + (beta/2) * ( pow(P[i][k],2) + pow(Q[k][j],2) )
        if e < 0.001:
            break
    return P, Q.T

###############################################################################

if __name__ == "__main__":
    R = Ratings

    #R = numpy.array(R)

    N = Ratings.shape[0]
    M = Ratings.shape[1]
    K = 20

    P = numpy.random.rand(N,K)
    Q = numpy.random.rand(M,K)

    nP, nQ = matrix_factorization(R, P, Q, K)

KeyboardInterrupt: 